# Spatial Interpolation

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (16, 9)


In [ ]:
import rasterio

In [ ]:
from rasterio.mask import mask
import geopandas as gpd
import fiona
import pandas as pd

In [ ]:
precip = gpd.read_file("precip_sd.geojson")


In [ ]:
precip.plot(column='inches', legend=True);

In [ ]:
county = gpd.read_file("sdcounty.geojson")

In [ ]:
m = county.explore()
precip.explore(column='inches', m=m)


## Interpolation Methods

### Voronoi Partition Interpolation

In [ ]:
from libpysal.cg import voronoi_frames
points = [(10.2, 5.1), (4.7, 2.2), (5.3, 5.7), (2.7, 5.3)]
regions_df, points_df = voronoi_frames(points)
regions_df.shape
(4, 1)

In [ ]:
regions_df.plot()

In [ ]:
#libpysal.cg.voronoi_frames(points, radius=None, clip='extent')
points = precip.get_coordinates().values

In [ ]:
points

In [ ]:
v_gdf, v_p = voronoi_frames(points)

v_gdf.plot()

In [ ]:
v_gdf, v_p = voronoi_frames(points, clip=county.geometry[0])

v_gdf.plot()

In [ ]:
v_gdf['inches'] = precip.inches

In [ ]:
v_gdf.plot(column='inches', legend=True)

In [ ]:
base = v_gdf.plot(column='inches', legend=True,
                 edgecolor='gray')
precip.plot(ax=base, color='w');

### Interpolate to Grids

- hex3 the county
- nearest neighbor interpolators
- 

In [ ]:
import tobler

In [ ]:
county_utm = county.to_crs(county.estimate_utm_crs())
precip_utm = precip.to_crs(precip.estimate_utm_crs())

In [ ]:
from tobler.util import h3fy


In [ ]:
county_h3 = h3fy(county_utm)

In [ ]:
county_h3.plot()

In [ ]:
m = county_h3.plot(color='grey')
precip_utm.plot(column='inches', ax=m, legend=True);

### Nearest neighbor

In [ ]:
hcents = county_h3.centroid


In [ ]:
m = hcents.plot(color='r')
precip_utm.plot(column='inches', ax=m, legend=True);

In [ ]:
gpd.sjoin_nearest(county_h3, precip_utm, distance_col="distances",
    lsuffix="left", rsuffix="right", exclusive=True)

In [ ]:
county_h3.shape

In [ ]:
county_h3['nn1_est'] = gpd.sjoin_nearest(county_h3, precip_utm, distance_col="distances",
    lsuffix="left", rsuffix="right", exclusive=True).inches

In [ ]:
county_h3.plot(column='nn1_est', legend=True);

### Knn5

In [ ]:
X = [[0], [1], [2], [3]]
y = [0, 0, 1, 1]
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X, y)
print(neigh.predict([[1.5]]))



In [ ]:
# Set number of neighbors to use
neighbors = 5

# Initialize KNN regressor
knn_regressor = KNeighborsRegressor(n_neighbors = neighbors, weights = "uniform") # no distance decay distance

# Fit to observed locations
knn_regressor.fit(precip_utm.get_coordinates(), precip_utm.inches)

In [ ]:
knn_regressor.predict(hcents.get_coordinates())


In [ ]:
county_h3['nn5_est'] = knn_regressor.predict(hcents.get_coordinates())


In [ ]:
county_h3.plot(column='nn5_est', legend=True);

In [ ]:
import matplotlib.pyplot as plt
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
county_h3.plot(column='nn1_est',ax=ax1)
county_h3.plot(column='nn5_est',ax=ax2)
ax1.set_title("knn=1")
ax2.set_title("knn=5");



In [ ]:
# Set number of neighbors to use
neighbors = 5

# Initialize KNN regressor
knn_regressor = KNeighborsRegressor(n_neighbors = neighbors, weights = "distance") # inverse distance weighting
# Fit to observed locations
knn_regressor.fit(precip_utm.get_coordinates(), precip_utm.inches)

In [ ]:
county_h3['nn5id_est'] = knn_regressor.predict(hcents.get_coordinates())


In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
county_h3.plot(column='nn5_est',ax=ax1)
county_h3.plot(column='nn5id_est',ax=ax2)
ax1.set_title("knn=5 (Unweighted)")
ax2.set_title("knn=5 (Inverse Distance Weights)");


## Next

In [ ]:
tracts = gdf.dissolve(by='TRACTCE20')

tracts.shape

### Surface to Area Interpolation

#### Spatial Join on Centroid

In [ ]:
cents = tracts.centroid

In [ ]:
cents.plot()

In [ ]:
type(cents)

In [ ]:
coord_list = [(x, y) for x, y in zip(cents.x, cents.y)]
tracts['centest'] = [x[0] for x in clipped.sample(coord_list)]
tracts.head()

In [ ]:
tracts['centroid'] = tracts.centroid
tracts.set_geometry('centroid', inplace=True)

In [ ]:
tracts.plot(column='centest', legend=True);

In [ ]:
tracts.set_geometry('geometry', inplace=True)
tracts.plot(column='centest', legend=True);

#### Zonal Methods of Surface to Area Interpolation

In [ ]:
import rasterstats

In [ ]:
gdf.head()

In [ ]:
tracts.plot()

In [ ]:
from rasterstats import zonal_stats
tstats = zonal_stats(tracts, "clipped_example.tif",
            stats="count min mean max median")

#elevations2 = zonal_stats(
#    sd_tracts.to_crs(dem.rio.crs),  # Geotable with zones
#    "../data/nasadem/nasadem_sd.tif",  # Path to surface file
#)
#elevations2 = pandas.DataFrame(elevations2)

In [ ]:
tstats[:5]

In [ ]:
tstats = pd.DataFrame(tstats)

In [ ]:
tstats.head()

In [ ]:
tstats.shape

In [ ]:
tracts.shape

In [ ]:
tracts['mean'] = tstats['mean'].values
tracts.plot(column='mean', legend=True);

In [ ]:
tracts['median'] = tstats['median'].values
tracts.plot(column='median', legend=True);

In [ ]:
tracts['range'] = tstats['max'].values - tstats['min'].values
tracts.plot(column='range', legend=True);

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(data=tracts, x='centest', y='mean')
plt.plot([10, 40], [10, 40]);

In [ ]:
sns.scatterplot(data=tracts, x='median', y='mean')
plt.plot([10, 40], [10, 40]);